# Convolutional Neural Networks Tutorial 1

This is a simple tutorial that will train the CIFAR10 dataset using a convolutional neural network  
Let's first start by importing the librairies that we need and then print the TensorFlow and Keras versions.  

__PS:__ It's better to run this on a **GPU**

In [0]:
import numpy as np
import tensorflow as tf

import datetime

from tensorflow.keras.datasets.cifar10 import load_data
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from PIL import Image, ImageOps

In [0]:
print("TensorFlow version: " + str(tf.__version__))
print("Keras version: " + str(tf.keras.__version__))

# Loading and Preprocessing

Let's start by loading the data from Keras dataset

The dataset is composed of:


*   50000 images for training.
*   10000 images for testing

Each image is a colour one of size 32x32.  
Data dimension is:
* X: (?, 32, 32, 3)

The labels are the class number to which a certain image belong to. There are 10 (hence CIFAR10):
* Airplane
* Automobile
* Bird
* Car
* Deer
* Dog
* Frog
* Horse
* Ship
* Truck

In [0]:
# Reading datasets
(x_train, y_train), (x_test, y_test) = load_data()


x_train = x_train / 255
x_test  = x_test / 255

print("X Train dimensions: " + str(x_train.shape))
print("Y Train dimensions: " + str(y_train.shape))

print("X Test dimensions: " + str(x_test.shape))
print("Y Test dimensions: " + str(y_test.shape))

In [0]:
# Printing a sample image
i=np.random.randint(0, x_train.shape[0] - 1)
plt.title('Title: Plotting image:%d .... Value is %d' % (i,y_train[i]))
#plt.imshow(np.squeeze(x_train_orig[i]), cmap='gray')
plt.imshow(x_train[i])
plt.show()

# Model

## Model Architecture
The model that we will use has the following layers:
* __Conv2D:__ `f=3, p=same, strides=1`
* __Conv2D:__ `f=3, p=0, strides=1`
* __MaxPooling2D:__ `f=2, strides=2`
* __Dropout:__ `rate=0.25`
* __Conv2D:__ `f=3, p=same, strides=1`
* __Conv2D:__ `f=3, p=0, strides=1`
* __MaxPooling2D:__ `f=2, strides=2`
* __Dropout:__ `rate=0.25`
* __Flatten__
* __Dense:__ `units=512`
* __Dropout:__ `rate=0.5`
* __Dense:__ `units=10`

## Layers' Number of Parameters
### Formula
The number of parameters in each layer is equal to the following:
* __Conv2D:__ `f x f x nb_filters_in_prev_layer x nb_filters + nb_filters`
* __MaxPooling2D:__  `0`
* __Dropout:__ `0`
* __Flatten:__ `0`
* __Dense:__ `size_of_input x size_of_output + size_of_output`

### Formula applied to our model
Hence, for our model the number of parameters are equal to:
* __Conv2D:__ `3 x 3 x 3 x 32 + 32 = 896
* __Conv2D:__ `3 x 3 x 32 x 32 + 32 = 9248`
* __MaxPooling2D:__ `0`
* __Dropout:__ `0`
* __Conv2D:__ `3 x 3 x 32 x 64 + 64 = 18496`
* __Conv2D:__ `3 x 3 x 64 x 64 + 64 = 36928`
* __MaxPooling2D:__ `0`
* __Dropout:__ `0`
* __Flatten:__ `0`
* __Dense:__ `2304 x 512 + 512 = 1180160` (check below to see how *2304* was calculated)
* __Dropout:__ `0`
* __Dense:__ `512 x 10 + 10 = 5130`

## Convolutional Layers' Output Sizes
### Formula
The output size for each layer is as follows:
* __Conv2D:__ `n_out = n_in in case padding is same or floor(((n_in + 2p -f) / s) + 1) otherwise`
* __MaxPooling2D:__ `n_out = floor(((n_in -f) / s) + 1)`

### Formula applied to our model
* __Conv2D:__ `n_out = 32` => The output is of size: `32 x 32 x 32`
* __Conv2D:__ `n_out = floor(((32 + 2*0 - 3) / 1) + 1) = 30` => The output is of size: `30 x 30 x 32`
* __MaxPooling2D:__ `n_out = floor(((30 - 2) / 2) + 1) = 15` => The output is of size: `15 x 15 x 32`
* __Conv2D:__ `n_out = 15` => The output is of size: `15 x 15 x 64`
* __Conv2D:__ `n_out = floor(((15 + 2*0 - 3) / 1) + 1) = 13` => The output is of size: `13 x 13 x 64`
* __MaxPooling2D:__ `n_out = floor(((13 - 2) / 2) + 1) = 6` => The output is of size: `6 x 6 x 64`

The flatten layer will then produce a input vector of size: `6 * 6 * 64 =` __2304__




In [0]:
nb_classes = np.amax(y_train) + 1

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', activation='relu', name='block1_conv1', input_shape=x_train.shape[1:]))
model.add(Conv2D(32, (3, 3), padding='valid', activation='relu', name='block1_conv2'))
model.add(MaxPooling2D(pool_size=(2, 2), name='block1_pool'))
model.add(Dropout(0.25, name='block1_dropout'))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu', name='block2_conv1'))
model.add(Conv2D(64, (3, 3), padding='valid', activation='relu', name='block2_conv2'))
model.add(MaxPooling2D(pool_size=(2, 2), name='block2_pool'))
model.add(Dropout(0.25, name='block2_dropout'))

model.add(Flatten())
model.add(Dense(512, activation='relu', name='fc'))
model.add(Dropout(0.5, name='dropout'))
model.add(Dense(nb_classes, activation='softmax', name='prediction'))

# Compilation

Next we need to compile the model by calling __model.compile()__ and specifying the following:
* __Optimizer__ We can choose stochastic gradient descent __SGD__ or any other more powerful optimisation method like __Adam__
* __Loss function__ The sparse_categorical_crossentropy
* __Metrics__ to use while training. Here the accuracy between the real y and the predicted one  

__model.summary()__ summarises the model showing the layers along with their parameters

In [0]:
model.compile(optimizer="Adam", loss='sparse_categorical_crossentropy', metrics=["accuracy"])

model.summary()

# Model Training

We train the model by calling __model.fit()__ and giving it the training x_train and y_train  
We can also validate the model by giving the x_test and y_test

The model will be trained and the parameters will be updated based on x_train and y_train **only**. x_test and y_test will be used to evaluate how well the model is doing on the validation set

In [0]:
validation = True
if validation is True:
  eval_data = (x_test, y_test)
else:
  eval_data = None
  
start = datetime.datetime.now()

history = model.fit(x =x_train, y = y_train, epochs=100, batch_size=1024, validation_data=eval_data)

end = datetime.datetime.now()
print("\nTraining took: " +str(end - start))

# Training Results

In [0]:
print ("Train Accuracy = %.4f"  % (history.history['acc'][-1]))
print ("Train Loss = %.4f"  % (history.history['loss'][-1]))

if validation is True:
  print ("Test Accuracy = %.4f"  % (history.history['val_acc'][-1]))
  print ("Test Loss = %.4f"  % (history.history['val_loss'][-1]))

# Plotting Results

Here we plot the training and evaluation metrics' history.
2 Plots:
* One for loss
* One for accuracy

In [0]:
  plt.figure(figsize=(12,5))
  plt.subplot(1, 2, 1)
  plt.plot(history.history['acc'], 'b', label='Training acc')
  if validation is True:
    plt.plot(history.history['val_acc'], 'r', label='Validating acc')
  plt.title('Model Accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend()

  plt.subplot(1, 2, 2)
  plt.plot(history.history['loss'], 'b', label="Training loss")
  if validation is True:
    plt.plot(history.history['val_loss'], 'r', label="Validating loss")
  plt.title('Model Loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend()
  
  plt.show()

# Predict 

Prepare an image containing any of the objects defined in CIFAR10:
* Airplane
* Automobile
* Bird
* Cat
* Deer
* Dog
* Frog
* Horse
* Ship
* Truck 

Change the name of the file in __imgname__ variable to be the full path to the image that you have prepared.  

If you are running this notebook in Google Colab and the image does not exist, it will prompt you to upload it.  
If you are running it locally, it will just prompt an error message telling you that the file is not found.

In [0]:
import os

try:
  from google.colab import files
except ImportError:
  pass

cifar10_labels = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck"
]

imgname = 'ship.jpg'

if os.path.exists(imgname) == False:
  try:
    files.upload()
  except:
    pass

try:  
  # Read Image, convert it to grayscale, resize it to 28x28, invert to in order to be white on black background
  img = Image.open(imgname)
  plt.title('Title: Plotting image ')
  plt.imshow(img)

  img = img.resize((32,32))

  # Transform it to an array and normalise it
  myimg = np.array(img)
  myimg = myimg / 255

  # Predict
  pred = model.predict(np.expand_dims(myimg, axis=0)).squeeze()
  print("Predictions for all categories:")

  for i in range(len(cifar10_labels)):
    print("{}: {:.2%}".format(cifar10_labels[i], pred[i]))

  prediction = np.argmax(pred, axis=-1)

  print("\n{:.2%} it's a {}".format(pred[prediction], cifar10_labels[prediction]))
except:
  print("File not found!!!")

# CPU vs GPU

If you were running this notebook on a GPU, switch to a CPU in order to see how slow a CPU is compared to GPU.  

__Hint:__ In Google Colab, you do this as follows:  
Go to Edit => Notebook Settings  and set __Hardware Accelerator__ to __None__


# Summary
In this tutorial, we learnt how to:
* Load and Preprocess data
* Create a Sequential Model
* Compile and view the model
* Train
* Visualise training results
* Test the model on an uploaded image
* Compare CPU and GPU training times